<a href="https://colab.research.google.com/github/jsrobson/2023-cis5450-termproject/blob/main/CIS5450_TermProject_01_acquire_data_COMPLETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook: Data Acquisition
This notebook captures the various efforts made by project team members with to acquire and parse data during the earliest phase of the project in late October and early November 2023 as the project proposal was being developed. These efforts were deprecated as new and better data sources were found. Nevertheless, these efforts reflect a foundational part of building the team's understanding of how to approach the work and merit **archival** retention.

**Note:** Each cell is annotated with a brief description of the code, its intended purpose, and its output. As these code pieces were intended to collect data from external sources or clean collected data for analysis and were part of larger codebases, they are not intended to be run within this notebook and are included for posterity only.

# Code Description

**Primary Data Parsing:** Prior to the discovery of an attribute-linked property dataset, several investigations were performed on [Price Paid data](https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads) maintained by the United Kingdom (UK) government to determine its suitability with respect to the project requirements. Two different approaches to data wrangling were trialled using Dask to read a large file (.csv format) of property data for England and Wales for the initial 2019 study period:
* selecting pre-filtered data for the initial 2019 study period and parsing further using hardcoded columnar names, and
* using XPath on the Price Paid data website to automatically read file headers into the file reading process.

These approaches were each trialled to determine their impact on the resulting dataframe. All were deprecated with the introduction of EPDC data acquisition and wrangling challenges, discussed further in **EPDC Data Acquisition**.

In [ ]:
import glob

# function reads property file to dask dataframe
def read_property_to_df(filepath, boroughs):
    # generate and sort list of filenames
    filenames = glob.glob(filepath + "/*.csv")
    filenames.sort()
    # initialize cols for dataframe
    cols = ['unique_id', 'price_paid', 'purchase_date', 'postcode',
            'property_type', 'new_build', 'estate_type', 'saon', 'paon',
            'street', 'locality', 'town', 'district', 'county',
            'transaction_category', 'ext']
    # initialize empty list of dataframes for consolidation
    dataframes = []
    # for each filename in list of filenames
    for filename in filenames:
        print("Parsing file ", filename)
        # get dask dataframe associated with filename
        file_in = dd.read_csv(filename, dtype='string', keep_default_na=False)
        # append dask dataframe to list
        dataframes.append(parse_property_segment(file_in, cols))
    # concatenate dataframes in list and reset index
    concat_df = dd.concat(dataframes).reset_index(drop=True)
    # return concatenated dataframe with index reset
    return concat_df.loc[concat_df['district'].str.lower().isin([
        borough.lower() for borough in boroughs])]

# function parses a given property dataframe with pre-set column names
def parse_property_segment(dataframe, col_names):
    # set dataframe column headers to col_names
    dataframe.columns = col_names
    # drop unneeded columns
    to_drop = ['transaction_category', 'ext']
    file_in = dataframe.drop(columns=to_drop)
    # cast price_paid as float
    file_in['price_paid'] = file_in['price_paid'].astype(float)
    # convert purchase_date as date/time
    file_in['purchase_date'] = dd.to_datetime(file_in['purchase_date'],
                                              format='%Y-%m-%d %H:%M')
    # get year, month, day as individual cols
    file_in['year'] = file_in['purchase_date'].dt.year.astype('int32')
    file_in['month'] = file_in['purchase_date'].dt.month.astype('int32')
    file_in['day'] = file_in['purchase_date'].dt.day.astype('int32')
    # return dataframe filtered by county being Greater London
    return file_in[file_in['county'] == 'GREATER LONDON']

In [ ]:
# download file tempo
!wget -nc http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019-part1.csv
#Webpage with info of table
# make request to target webpage
w = requests.get("https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd")
# initialize dom tree object
dom_tree = html.fromstring(w.content)
# instantiate identified xpath string
table_body_xpath  = "/html/body/div[3]/main/div[4]/div[1]/div/div[3]/div/table/tbody"

# Get headers for csv using xpath
headers = dom_tree.xpath(table_body_xpath + "/tr/th/text()")
# place headers in list
cleaned_headers = [item.strip() for item in headers if item.strip()]

# Get description of header
description_header = dom_tree.xpath(table_body_xpath + "/tr")

# build headers list
description_header_list = []
for row in description_header:
    row_data = row.xpath("td//text()")
    description_header_list.append(row_data)

# Create a DataFrame of Header and description
header_df = pd.DataFrame({'Header': cleaned_headers, 'Description': description_header_list})

# Read the CSV file with the header
price_paid_df = pd.read_csv("pp-2019-part1.csv", header=None, names=cleaned_headers)

**Secondary Data Parsing**: As part of early data investigations, public and open source data for [private](https://data.london.gov.uk/dataset/local-authority-average-rents) and [social](https://data.london.gov.uk/dataset/local-authority-average-rents) housing rents were sourced as .csv from the London Datastore (Mayor of London / London Assembly), a central repository for urban data pertaining for Greater London.

It was collectively decided that purchase price exhibited a more direct relationship to the project problem statement. At time of decision, the team had written scratch methods to read and parse the specified files.



In [ ]:
# helper method reads and parses rent data for private housing stock, by borough
# returns pandas dataframe
def read_rents_private_df():
    # assign dtypes to each column
    d_rents = {
        "borough": "str",
        "lower_quartile": "int32",
        "median": "int32",
        "upper_quartile": "int32"
    }
    # read file
    file_in = pd.read_csv("resources/ldn_rent2023.csv",
                          dtype=d_rents)
    # standardize borough names by swapping ampersand with and
    file_in['borough'] = file_in['borough'].str.replace("&", "and")
    return file_in


# helper method reads and parses rent data for social housing stock, by borough
# returns pandas dataframe
def read_rents_social():
    # read file
    file_in = pd.read_csv("resources/ldn_rentsoc.csv")
    # replace any incomplete element with none
    file_in = file_in.replace({'LSVT': None, '..': None, '.': None})
    # drop nulls and unnecessary columns
    file_in.dropna(inplace=True)
    file_in.drop(axis=1, columns=['Code', 'New Code'], inplace=True)
    # get column names not related to area
    columns = [i for i in file_in.columns if i not in ['Area']]
    # convert non-string columns to float
    for col in columns:
        file_in[col] = pd.to_numeric(file_in[col], errors='ignore')
    # rename column for common alignment
    file_in.rename(inplace=True, columns={'Area': 'borough'})
    return file_in

**EPDC Data Acquisition**: While the Price Paid dataset is an important and rich dataset, it lacks a crucial means of standardization: a corresponding floor area for each listed property to derive a relative cost-per-square-metre calculation.

The UK Government also maintains stores of [environmental performance (EPDC) data](https://www.gov.uk/government/statistical-data-sets/live-tables-on-energy-performance-of-buildings-certificates) for buildings that are updated whenever a building in the UK is leased or sold. Crucially, the data includes a square metre feature `tfarea`.

Though the data is freely available, it is limited to 5000 records per query when using the web-based querying tool. This motivated a need to pursue alternative code-based approaches to data acquisition using the Python [requests](https://requests.readthedocs.io/en/latest/) library. The team wrote methods to access EPDC data batched by quarter and year for each borough in Greater London. Preliminary work was performed on the collected data to join with housing purchase records wherever possible.

The discovery of an [attribute-linked dataset](https://reshare.ukdataservice.ac.uk/854942/) prepared by University College London (UCL) researchers that joined these two disparate datasets limited the utility of this work. As our joined dataset exhibited drastically fewer rows after the merge of purchase price and environmental performance data, it was decided to use the more stable UCL dataset release.


In [ ]:
# import statements
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd


# code gets UK domestic environmental performance data for housing
# from web server using Python requests library
# (https://requests.readthedocs.io/en/latest/)

# helper method uses existing data source to build
# dict of k: borough / v: code (i.e. numeric code corresponding to borough)
def borough_dict():
    # create dataframe from social rents
    df = pd.read_csv('resources/ldn_rentsoc.csv')
    # get borough names and codes as lists
    boroughs = df['Area'].tolist()
    codes = df['New Code'].tolist()
    # create dictionary from borough name (k) - code (v) pair
    return {boroughs[i]: codes[i] for i in range(len(boroughs))}


# helper method initializes time segment data structures to
# organize data gained from web
def time_ds():
    # initialize year and month ranges
    years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
             '2019', '2020', '2021', '2022']
    batch = [('1', '3'), ('4', '6'), ('7', '9'), ('10', '12')]
    # return as tuple
    return years, batch


# main method, get epdc data using requests library
def main():
    # get borough codes, years, and month ranges
    b_codes = borough_dict()
    years, batch = time_ds()
    # initialize api key (scrubbed for public view)
    key = '<REMOVED>'
    # use requests basic authentication
    basic = HTTPBasicAuth('jsrobson@seas.upenn.edu', key)
    # set headers
    headers = {'Accept': 'text/csv'}
    # initialize url stub and related text strings
    url_stub = 'https://epc.opendatacommunities.org/api/v1/domestic/search'
    l_auth = 'local-authority='
    year_f = ('from-year=', 'from-month=')
    year_t = ('to-year=', 'to-month=')
    # for each borough in dict, corresponding code
    for borough, code in b_codes.items():
        # for each year in 2011 - 2022
        for year in years:
            # print message for reference
            print("Getting " + year + " for " + borough)
            # for each bi-annual batch
            for tup in batch:
                # generate url stub
                url = (url_stub + '?' + l_auth + code + '&' + year_f[1] +
                       tup[0] + '&' + year_f[0] + year + '&' + year_t[1] +
                       tup[1] + '&' + year_t[0] + year + '&size=10000')
                # get url using requests
                r = requests.get(url, auth=basic, headers=headers)
                # generate filename title
                title = (borough + '_' + year + '_' + tup[0] + '_' + tup[1] +
                         '.csv')
                # print url for reference
                print(url)
                # # save data as csv at title
                open('output/' + title, 'wb').write(r.content)


if __name__ == '__main__':
    main()

**EPDC Data Cleaning:** Once the EPDC data was collected using the above requests-driven process, two functions were written to read the data from .csv format into a Pandas dataframe and parse it according to best practices.

This code was used to test initial viability of including EPDC data in the overall project dataset and, as above, was deprecated when the UCL attribute-linked dataset was sourced.

In [ ]:
import glob

# function reads EPDC data at filepath into pandas dataframe,
# returns cumulative EPDC dataframe using helper function
def read_epdc_to_df(filepath):
    # generate and sort list of filenames
    filenames = glob.glob(filepath + "/*.csv")
    filenames.sort()
    # initialize empty list of dataframes for consolidation
    dataframes = []
    # for each available file
    for filename in filenames:
        # append the parsed dataframe chunk into array of df
        dataframes.append(parse_epdc_segment(filename))
    # return concatenated cumulative df from chunked dfs
    return dd.concat(dataframes).reset_index(drop=True)


# function reads EPDC data segment at filepath into pandas dataframe
# and parses according to anticipated uses for data and best practices
def parse_epdc_segment(filepath):
    # generate dataframe from csv at path
    file_in = dd.read_csv(filepath, dtype='string', keep_default_na=False)
    # identify columns to keep
    to_keep = ['lmk-key', 'address1', 'address2', 'address3', 'postcode',
               'property-type', 'built-form', 'local-authority', 'tenure',
               'total-floor-area', 'uprn']
    # drop unneeded columns
    file_in = file_in[to_keep]
    # drop any column where uprn is empty
    file_in = file_in[file_in['uprn'] != '']
    # cast total-floor-area as float and uprn as int
    file_in['total-floor-area'] = file_in['total-floor-area'].astype(float)
    file_in['uprn'] = file_in['uprn'].astype(int)
    # return dataframe
    return file_in.sort_values(by='uprn', ascending=True)

**Developing an appropriate join key:** With both price paid and EPDC data collected, it became a matter of determining how to join the data such that a property with price paid data and floor area data could be united in one location. Initially, the address of the property was considered as an obvious solution. However, an address across the two datasets was not consistent. For example, EPDC data for the London borough of Camden in 2019 contains an address in the format:


```
address1: Flat 86
address2: 19-23 Fitzroy Street
address3: ''
```
while the same address in the purchase paid data is arrayed as:

```
address1: 19-23
address2: FLAT 86
address3: FITZROY STREET
```
There were also inconsistencies among address series within each dataset such that resolving the different address representations across datasets was not guaranteed to yield the correct result.

Mercifully, supplemental research on this issue yielded a possible solution: a [mapping](https://ubdc.ac.uk/data-services/data-catalogue/housing-data/price-paid-data-to-uprn-lookup/) of unique purchase identifier `transactionid` within purchase price data to unique property reference number `uprn` within EPDC data. This mapping was created by the [Urban Big Data Centre](https://www.ubdc.ac.uk/) (UK) using price paid and EPDC datasets for a time period of October 2008 to 2021.

Code was written to read the mapping, presented in .csv format, into a dataframe for use in a dataframe merge process. The overhead to process this data motivated the search for a different approach. As above, this approach was deprecated when the UCL attribute-linked dataset was sourced.



In [ ]:
from dask import dataframe as dd

# helper function read the property mapping (uprn, transaction_id)
# into a pandas dataframe
def read_pkey_to_df(filename):
    # allocate appropriate datatypes in dict
    types = {
        'uprn': 'int32',
        'transactionid': 'string'
    }
    # read csv document filename into dask dataframe
    p_key = dd.read_csv(filename, dtype=types)
    # return only the key mapping between uprn (EPDC) and transactionid (Ppaid)
    return p_key[['uprn', 'transactionid']]


# given df of properties and of keys, conduct merge to integrate uprn value
# rerturns merged pandas df
def merge_properties_keys_df(props, keys):
    # merge properties df and keys df with matching transactionid
    props_m = props.merge(keys, left_on='unique_id',
                          right_on='transactionid', how='left')
    # drop transactionid as redundant
    props_m = props_m.drop(columns=['transactionid'])
    # drop any extant nulls
    props_m = props_m.dropna()
    # cast uprn value as integer type and return
    props_m['uprn'] = props_m['uprn'].astype(int)
    return props_m

**Large file handling:** Once the UCL attribute-linked dataset was discovered, a new challenge was presented. The dataset contained our desired price paid and environmental performance data but accounted for a larger time period and geographic scale than strictly required for the problem definition, and so with it, a file size exceeding 6.0 GB.

The team developed code to read the given dataset as a Dask dataframe, conduct preliminary filtering and parsing activities to control for geography and time, and save the resulting output in .csv format delimited by the updated study period of 2017-2019. This ensured that we were not working with a larger dataset than was ultimately necessary. Dask was used to quickly read the given .csv file using parallelism; the dataframe was converted to Pandas once major parallel operations were performed.

This code is included as it supported creation of a new, smaller, and controlled dataset and is not persistently re-used.

In [ ]:
# import statements
from lh_data import read_boroughs_to_list
from dask import dataframe as dd


# main method
def main():
    # call helper method to read large file and output csv
    read_large_file()


# method reads large data file (> 6.0GB) and filters for
# project parameters to reduce size.
def read_large_file():
    # get boroughs list
    boroughs = read_boroughs_to_list('resources/ldn_borough.csv')
    # convert boroughs to lowercase
    boroughs = [borough.lower() for borough in boroughs]
    # set dtypes to silence warning
    dtype = {
        'CO2_EMISS_CURR_PER_FLOOR_AREA': 'float64',
        'LOW_ENERGY_LIGHTING': 'float64',
        'MAIN_HEATING_CONTROLS': 'string',
        'MULTI_GLAZE_PROPORTION': 'float64',
        'price': 'float64',
        'UPRN': 'float64'
    }
    # read target large file csv as dask dataframe
    file = dd.read_csv("resources/tranall2011_19.csv", dtype=dtype,
                       assume_missing=True)
    # get pandas dataframe using parsing helper method
    df = parse_large_file(file)
    # filter by inner london areas defined by boroughs ds
    df = df.loc[df['district'].str.lower().isin(boroughs)]
    # filter by year range 2017 - 2019
    df = df[(df['year'] == 2017) | (df['year'] == 2018) | (df['year'] == 2019)]
    # export dataframe to csv
    df.to_csv('out.csv')


# given a dask df derived from large csv file, helper method
# parses and filters as needed and returns converted pandas df.
def parse_large_file(df):
    # convert data of transfer to_datetime file
    df['dateoftransfer'] = dd.to_datetime(df['dateoftransfer'],
                                          format='%Y-%m-%d')
    # get year and convert to integer
    df['year'] = df['dateoftransfer'].dt.year.astype('int32')
    # build conditional statement where county contains London
    cond_place = df['county'].str.contains('LONDON', case=False)
    # get filtered dataframe given condition
    f_df = df[cond_place]
    # return dask dataframe as pandas df
    return f_df.compute()


if __name__ == '__main__':
    main()

**SQL database development:** As a corollary to large file handling issues, the team weighted the merits and utility of using a structured query language (SQL) and relational database approach to data storage and management. As we continued to work with the data and introduce Dask for parallelism, it became clear that an SQL-driven approach would not ultimately be necessary and so the decision was made to maintain a Pandas environment.

The below code represents preliminary SQL database development and the later creation of a function as a means to encapsulate the same effort.

In [ ]:
# Uba's SQL work
!pip install db-sqlite3

# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


# Connect to SQLite database
# to do: update path
conn = sqlite3.connect('/content/gdrive/My Drive/temp.db')

# Save the DataFrame to a table in the SQLite database
price_paid_df.head(20).to_sql('tmep_tab', conn, index=False, if_exists='replace')

# sql query to test
query = "SELECT Price FROM tmep_tab"

result_df = pd.read_sql_query(query, conn)

# Close connection
conn.close()
result_df

In [ ]:
# condensing Uba's work into function that can be called to return df from sql db
def convert_df_to_sql_db(db_filepath, dataframe, query):
    # create a connection to database
    connect = sqlite3.connect(db_filepath)
    # convert passed df to sql db
    dataframe.to_sql('temp_table', connect, index=False, if_exists='replace')
    # pass query and store result as dataframe
    result_df = pd.read_sql_query(query, connect)
    # close database connection and return result_df
    connect.close()
    return result_df